This notebook is intended to be a short tutorial on opening, viewing, and manipulating point clouds using python.  We will first read the file in using the laspy package, and then extract its points.  Then for ease of use, the points in the cloud will be converted into an open3d point cloud as seene below.  

In [3]:
import laspy
import numpy as np
import open3d as o3d


las = laspy.read("src/dataExploration/notebook/CodyKrest9_4_24_riesling_chest1_ref6.las")

points = np.vstack((las.x, las.y, las.z)).transpose()

r = las.red/255
g = las.green/255
b = las.blue/255
rgb_colors = np.vstack((r, g, b)).transpose()

if points.shape[0] == 0:
    print("Error: No points found in LAS file!")
else:
    print(f"Successfully loaded {points.shape[0]} points.")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(rgb_colors)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Successfully loaded 31117012 points.


For large files, the data a long time to load.  For the sake of this tutorial, the point cloud has been downsampled it 1/20th of its original size using the code below.  

In [6]:
o3d.io.write_point_cloud("converted.ply", pcd)
voxel_size = 0.05
pcd_down = pcd.voxel_down_sample(voxel_size)

Now that the code has been loaded and down sampled, one can simple call the draw_geometries function to create the 3D image from the cell below.  You can scroll in and out to increase and reduce the zoom, as well as click and drag to rotate the image.  This cell will continue running until the image tab is closed.  

In [8]:
o3d.visualization.draw_geometries([pcd_down])

Below are some transformations that can be used to change the image for better viewing.  

In [11]:
pcd_down.translate((0, 2, 0)) 
R = pcd_down.get_rotation_matrix_from_xyz((0, np.pi/2, 0))  
pcd_down.rotate(R, center=(0, 0, 0))
pcd_down.scale(2.0, center=(0, 0, 0))

PointCloud with 18282324 points.

Once again visualizing the file after the translations to see a difference

In [14]:
o3d.visualization.draw_geometries([pcd_down])

Here you can filter to remove any statistical outliers.

In [16]:
cl, ind = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_filtered = pcd_down.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_filtered])

An easy way to create a mesh can be done as such

In [20]:
pcd_filtered.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_filtered, depth=9)
o3d.visualization.draw_geometries([mesh])


Now that you know how to load, open, manipulate, and create a mesh from the file, you can export it as seen below:

In [22]:
o3d.io.write_point_cloud("processed.ply", pcd_filtered)

True